In [749]:
# Install all the necessary python libraries if anaconda is not able to find them.
# !pip3 install google-maps-reviews
# Too bad the google-maps-reviews lib requires OutScraper API Key which is only free for the first 500 places and it requires minimum deposit of $500

In [750]:
# Import all the necessary python libraries
# import requests
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException


In [751]:
# Note: By default, The Safari WebDriver is included with the operating system. There is no additional installations required.
driver = webdriver.Safari()
driver.maximize_window()
url = 'https://www.google.com/maps'
driver.get(url)
time.sleep(5)

In [752]:
# Functions to return the element

# def find_web_element(locator_strategy="XPATH",locator_value=None,child_driver=None,iterate=False):

#     # Map string locator strategy to By class attributes
#     locator_strategy_map = {
#         'XPATH': By.XPATH,
#         'CSS': By.CSS_SELECTOR,
#         'ID': By.ID,
#         'NAME': By.NAME,
#         'CLASS': By.CLASS_NAME,
#         'TAG': By.TAG_NAME,
#         'LINK': By.LINK_TEXT,
#         'PARTIAL': By.PARTIAL_LINK_TEXT,
#     }
    
#     current_driver = child_driver or driver
#     # Default to By.XPATH if not found
#     by_locator = locator_strategy_map.get(locator_strategy.upper(), By.XPATH)
#     print(by_locator)

#     if iterate is False:
#         return current_driver.find_element(by_locator, locator_value)
#     else:
#         return current_driver.find_elements(by_locator, locator_value)

# def find_element_by_css(xpath):
#     return driver.find_element(By.XPATH, xpath)

# def find_element_by_class(class_name):
#     return driver.find_element(By.CLASS_NAME, class_name)

In [753]:
# Define CONSTANTS
FOOD_CATEGORY_LIST = ['halal food','western food','fast food','vegetarian food',
                      'chinese food','indian food','malay food','japanese food',
                      'korean food','cafe','bakery']

DRINK_CATEGORY_LIST = ['coffee','bubble tea']

LOCATION_LIST = ['Subang Jaya','Puchong','Sri Petaling','Bukit Jalil','Seri Kembangan','Bukit Bintang','Old Klang Road','Kepong','Klang','Cheras']

SEARCH_BOX_INPUT_XPATH = '//*[@id="searchboxinput"]'
CLOSE_BUTTON_XPATH = '//*[@id="searchbox"]/div[2]/button'
SCROLLABLE_COMPONENT_XPATH = '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]'
LOADING_SPINNER_XPATH = '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[146]/div/div'
END_OF_LIST_XPATH = '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[169]/div/p/span'
RESULT_LABEL_XPATH = '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[2]/div[2]/div[1]/h1'
RESULT_CLASS_NAME = 'hfpxzc'
RESULT_LIST_XPATH = '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]'

FNB_OPENING_HOURS_BTN_DOWN_XPATH = '//*[@aria-expanded="false" and @role="button" and @tabindex="0" and @jsaction[starts-with(., "pane.openhours")]]'
FNB_OPENING_HOURS_BTN_RIGHT_XPATH = '//*[@data-item-id="oh"]'
FNB_OPENING_HOURS_DETAILS_XPATH = '//*[@aria-label[contains(., "Hide open hours for the week")]]'
FNB_OPENING_HOURS_CLOSED_CLASS_NAME = ' fCEvvc' #Temporarily Closed / #Permanently Closed
FNB_OPENING_HOURS_CLOSED_XPATH = '//span[@class=" fCEvvc"]' #'//*[@jslog[starts-with(., "75719")]]' #Temporarily Closed / #Permanently Closed
FNB_ADDRESS_XPATH = '//*[@data-item-id[starts-with(., "address")]]'
FNB_PHONE_XPATH = '//*[@data-item-id[starts-with(., "phone")]]'
FNB_MENU_XPATH = '//*[@data-item-id[starts-with(., "menu")]]'
FNB_WEBSITE_XPATH = '//*[@data-item-id[starts-with(., "authority")]]'
FNB_ORDER_XPATH = '//*[@aria-label[starts-with(., "Place an order")]]'

In [754]:
# Initialize Variables
# df = pd.DataFrame()#columns=['Name'])
fnb_list = []
result_element = []

In [755]:
def perform_search(search_input):

    # Locate the search box element
    search_box_element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, SEARCH_BOX_INPUT_XPATH))
    )
    search_box_element.clear()
    time.sleep(0.5)
    search_box_element.send_keys(search_input + Keys.ENTER)
    # Scrolling the bar here
    # Reference: https://stackoverflow.com/questions/72543054/how-to-scroll-down-google-maps-using-selenium-python
    # side_bar_element = WebDriverWait(driver, 10).until(
    #     EC.presence_of_element_located((By.XPATH, RESULT_LIST_XPATH))
    # )

    keepScrolling=True
    while(keepScrolling):
        side_bar_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, RESULT_LIST_XPATH))
        )
        side_bar_element.send_keys(Keys.PAGE_DOWN)
        time.sleep(0.5)
        side_bar_element.send_keys(Keys.PAGE_DOWN)
        time.sleep(0.5)
        end_of_list_element = driver.find_element(By.TAG_NAME, "html").get_attribute('outerHTML')
        if(end_of_list_element.find("You've reached the end of the list.") != -1):
            keepScrolling = False

    # result_label_element = WebDriverWait(driver, 10).until(
    #     EC.element_to_be_clickable((By.XPATH, RESULT_LABEL_XPATH))
    # )
    # result_label_element.click()

    # scrollable_element = WebDriverWait(driver, 10).until(
    #     EC.element_to_be_clickable((By.XPATH, SCROLLABLE_COMPONENT_XPATH))
    # )
    # scrollable_element.click()
    # #
    # scrollable_element.send_keys(Keys.PAGE_DOWN)
    # end_of_list = False

    # while end_of_list is False:
    #     try:
    #         end_of_list_element = driver.find_element(By.XPATH, END_OF_LIST_XPATH)
    #         if end_of_list_element:
    #             end_of_list = True
    #     except:
    #         print("going to scroll")
    #         scrollable_element.send_keys(Keys.PAGE_DOWN)
    #         WebDriverWait(driver, 10).until(
    #             EC.invisibility_of_element_located((By.XPATH, LOADING_SPINNER_XPATH))
    #         )
    #         time.sleep(2)
#iterate_category(DRINK_CATEGORY_LIST, LOCATION_LIST)


In [756]:
def convert_opening_hours_to_dict(opening_hours):

    day_parts = opening_hours.replace('\u202f', ' ').replace('. Hide open hours for the week', '').split(';')

    days_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

    open_hours_dict = {
        part.split(',')[0].strip(): part.split(',', 1)[1].strip()
        for part in day_parts
    }

    sorted_open_hours_dict = {day: open_hours_dict[day] for day in days_order if day in open_hours_dict}

    return sorted_open_hours_dict


In [757]:
def get_fnb_details(result_element):
    for fnb in reversed(result_element):               
        name = fnb.get_attribute('aria-label')
        google_map_url = fnb.get_attribute('href')
        
        fnb.click()
        time.sleep(1)

        opening_hours = ''
        phone = ''
        menu_url = ''
        website_url = ''
        order_url = ''
        address = fnb.find_element(By.XPATH, FNB_ADDRESS_XPATH).text
    
        try:
            # CASE 1: OPENING HOURS BUTTON IS POINTING DOWN
            opening_hours_btn_elem = fnb.find_element(By.XPATH, FNB_OPENING_HOURS_BTN_DOWN_XPATH)
            time.sleep(1)
            opening_hours_btn_elem.click()
            time.sleep(1)
            opening_hours_elem = fnb.find_element(By.XPATH, FNB_OPENING_HOURS_DETAILS_XPATH)
            opening_hours = opening_hours_elem.get_attribute('aria-label')
            opening_hours_dict = convert_opening_hours_to_dict(opening_hours)

        except NoSuchElementException:

            # CASE 2: OPENING HOURS BUTTON IS POINTING RIGHT
            try:
                opening_hours_btn_elem = fnb.find_element(By.XPATH, FNB_OPENING_HOURS_BTN_RIGHT_XPATH)
                time.sleep(1)
                opening_hours_btn_elem.click()
                time.sleep(1)
                opening_hours_elem = fnb.find_element(By.XPATH, FNB_OPENING_HOURS_DETAILS_XPATH)
                opening_hours = opening_hours_elem.get_attribute('aria-label')
                opening_hours_dict = convert_opening_hours_to_dict(opening_hours)

            except NoSuchElementException:

                # CASE 3: FNB IS TEMPORARILY CLOSED OR PERMANENTLY CLOSED
                try:
                    opening_hours_dict = fnb.find_element(By.XPATH, FNB_OPENING_HOURS_CLOSED_XPATH).text

                except NoSuchElementException:
                    
                    # CASE 4: NO OPENING HOURS STATED
                    opening_hours_dict = 'Unknown'
                pass
            pass

        try:
            phone = fnb.find_element(By.XPATH, FNB_PHONE_XPATH).text
        except NoSuchElementException:
            phone = None
            pass

        try:
            menu_url = fnb.find_element(By.XPATH, FNB_MENU_XPATH).get_attribute('href')
        except NoSuchElementException:
            menu_url = None
            pass 

        try:
            website_url = fnb.find_element(By.XPATH, FNB_WEBSITE_XPATH).get_attribute('href')
        except NoSuchElementException:
            website_url = None
            pass 
        
        try:
            order_url = fnb.find_element(By.XPATH, FNB_ORDER_XPATH).get_attribute('href')
        except NoSuchElementException:
            order_url = None
            pass 

        fnb_details = {
            "Name": name,
            "Address": address,
            "Opening Hours": opening_hours_dict,
            "Phone": phone,
            "Google Map Url": google_map_url,
            "Menu Url": menu_url,
            "Website Url": website_url,
            "Order Url": order_url
        }
        print(fnb_details)
        #global fnb_list
        fnb_list.append(fnb_details)

In [758]:
def iterate_category(cateogry_list, location_list):
    for category in cateogry_list:
        for location in location_list:
            search_input = f'{category} near {location}'
            perform_search(search_input)
            time.sleep(3)
            #print('going to find result_element')
            #result_element = WebDriverWait(driver, 10).until(
            #    EC.presence_of_all_elements_located((By.CSS_SELECTOR,f"div[aria-label='Results for {search_input}']"))
            #)
            global result_element
            result_element = driver.find_elements(By.CLASS_NAME, RESULT_CLASS_NAME)
            #print(type(result_element))
            print(len(result_element))
            
            get_fnb_details(result_element)
            break
        break

In [759]:
iterate_category(FOOD_CATEGORY_LIST, LOCATION_LIST)
iterate_category(DRINK_CATEGORY_LIST, LOCATION_LIST)


65
{'Name': 'SY IMPERIAL (HALAL DIM SUM & STEAMBOAT RESTAURANT)', 'Address': '2A,NADAYU 28,DAGANG, Jalan PJS 11/7, Bandar Sunway, 47500 Subang Jaya, Selangor', 'Opening Hours': 'Temporarily closed', 'Phone': '011-7018 4299', 'Google Map Url': 'https://www.google.com/maps/place/SY+IMPERIAL+%28HALAL+DIM+SUM+%26+STEAMBOAT+RESTAURANT%29/data=!4m7!3m6!1s0x31cc4d1836890c5d:0x113c885985f63d1c!8m2!3d3.0706879!4d101.6026793!16s%2Fg%2F11s592r4zx!19sChIJXQyJNhhNzDERHD32hVmIPBE?authuser=0&hl=en&rclk=1', 'Menu Url': None, 'Website Url': 'https://letsumai.com/widget/sy-imperial', 'Order Url': None}
{'Name': 'Desiran Cafe', 'Address': 'Jalan USJ 25/1F, One City, 47630 Subang Jaya, Selangor', 'Opening Hours': 'Temporarily closed', 'Phone': '017-499 4105', 'Google Map Url': 'https://www.google.com/maps/place/Desiran+Cafe/data=!4m7!3m6!1s0x31cdb3248cc0bddd:0xdfc49e83742b1e16!8m2!3d3.0212724!4d101.5792579!16s%2Fg%2F11cr_xlphh!19sChIJ3b3AjCSzzTERFh4rdIOexN8?authuser=0&hl=en&rclk=1', 'Menu Url': None, 'Web

KeyboardInterrupt: 

In [ ]:
fnb_list = []
get_fnb_details(result_element)

{'Name': 'SY IMPERIAL (HALAL DIM SUM & STEAMBOAT RESTAURANT)', 'Address': 'Lot 53 Summit USJ Subang Jalan Kewajipan USJ 1, Taman Perindustrian Uep, 47600 Subang Jaya, Selangor', 'Opening Hours': {'Monday': '10 am to 10 pm', 'Tuesday': '10 am to 10 pm', 'Wednesday': '10 am to 10 pm', 'Thursday': '10 am to 10 pm', 'Friday': '10 am to 10 pm', 'Saturday': '10 am to 10 pm', 'Sunday': '10 am to 10 pm'}, 'Phone': '011-7000 5646', 'Google Map Url': 'https://www.google.com/maps/place/SY+IMPERIAL+%28HALAL+DIM+SUM+%26+STEAMBOAT+RESTAURANT%29/data=!4m7!3m6!1s0x31cc4d1836890c5d:0x113c885985f63d1c!8m2!3d3.0706879!4d101.6026793!16s%2Fg%2F11s592r4zx!19sChIJXQyJNhhNzDERHD32hVmIPBE?authuser=0&hl=en&rclk=1', 'Menu Url': None, 'Website Url': 'http://www.johnnyrestaurant.com/', 'Order Url': 'https://www.foodpanda.my/restaurant/s2gl/johnnys-restaurant-summit-usj?utm_source=google&utm_medium=organic&utm_campaign=google_reserve_place_order_action'}


IndexError: list index out of range

In [ ]:
driver.quit()


In [760]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
df = pd.DataFrame(fnb_list)
df['Opening Hours']


0                                                                                                                                                                                                                                 Temporarily closed
1                                                                                                                                                                                                                                 Temporarily closed
2                                                                                                                                                                                                                                 Temporarily closed
3                                                                                                                                                                                                                                 Permanently closed
4    {'Monday': '11:

In [ ]:
text = df['Opening Hours'].iloc[2]